# Example bulk data client interaction

Implements a simple [FHIR bulk data client](http://wiki.hl7.org/index.php?title=201801_Bulk_Data) with authentication according to [SMART Authorization Guide protocol](http://docs.smarthealthit.org/authorization/backend-services/).

### Config

Start by reading required config parameters. We'll test against https://bulk-data.smarthealthit.org.

In [1]:
import yaml

with open('config.yaml') as f:
    config = yaml.load(f)

### Setup

Construct the client. First we request the remote server to prepare a dataset using ``client.provision()``

In [2]:
from fhir.client import BulkDataClient

client = BulkDataClient(**config)
client.provision()

We can check for successful provisioning of the list of data files using a property of the client

In [3]:
client.provisioned

True

We could also provision data including [valid bulk data request parameters](https://github.com/smart-on-fhir/fhir-bulk-data-docs/blob/master/README.md#query-string-parameters)

In [4]:
client.provision(_since='2010-01-28')

### Consume data

Once the dataset is provisioned we can iterate through the resources in the bulk data set. The client leaves most data  on the remote server and waits to retrieve subsequent files until the previous file in the manifest is consumed.

In [5]:
data = client.iter_json()
next(data)

{u'activity': [{u'detail': {u'code': {u'coding': [{u'code': u'226234005',
       u'display': u'Healthy diet',
       u'system': u'http://snomed.info/sct'}]},
    u'status': u'in-progress'}},
  {u'detail': {u'code': {u'coding': [{u'code': u'703993001',
       u'display': u'Colonoscopy planned',
       u'system': u'http://snomed.info/sct'}]},
    u'status': u'in-progress'}},
  {u'detail': {u'code': {u'coding': [{u'code': u'243072006',
       u'display': u'Cancer education',
       u'system': u'http://snomed.info/sct'}]},
    u'status': u'in-progress'}}],
 u'addresses': [{u'reference': u'urn:uuid:d4f83046-5f89-4378-96f4-4e15766f9d97'}],
 u'category': [{u'coding': [{u'code': u'395082007',
     u'display': u'Cancer care plan',
     u'system': u'http://snomed.info/sct'}]}],
 u'context': {u'reference': u'urn:uuid:334228ee-2228-43c4-bea3-438e83d73017'},
 u'intent': u'order',
 u'period': {u'start': u'2015-11-09'},
 u'resourceType': u'CarePlan',
 u'status': u'active',
 u'subject': {u'reference':

### Cleanup

We should close the connection in the underlying session prior to releasing the client. Alternatively the client also functions as a context manager for simplicity.

In [6]:
client.session.close()

with BulkDataClient(**config) as client:
    client.provision()
    print client.provisioned

True
